# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [15]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = '../data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])


In [16]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [17]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
def experiment(regression,train,test,divided_by=5):
    Re = regression()
    start = time.time()
    print("accuracy:{}".format(cross_val_score(Re,train,test,cv=divided_by).mean()))
    print("time:{}".format(time.time()-start))


df_compare = pd.DataFrame()
for col in df.columns:
    df_compare[col] = LabelEncoder().fit_transform(df[col])
print(df_compare.head())
experiment(LogisticRegression,df_compare[:train_num],train_Y)

   Name  Sex  Ticket  Cabin  Embarked
0   155    1     720    185         3
1   286    0     816    106         0
2   523    0     914    185         3
3   422    0      65     70         3
4    22    1     649    185         3
accuracy:0.780004837244799
time:0.03293728828430176


In [37]:
# 均值編碼 + 邏輯斯迴歸

def mean_encoding_transform(data,target="Survived"):
    for c in df.columns:
        mean_df = data.groupby([c])[target].mean().reset_index()
        mean_df.columns = [c, f'{c}_mean']
        data = pd.merge(data, mean_df, on=c, how='left')
        data = data.drop([c] , axis=1)
    data = data.drop([target] , axis=1)
    return data
    
df_train2 = pd.concat([df_compare[:train_num], train_Y], axis=1)

new_df = mean_encoding_transform(df_train2)
print(new_df.head())
experiment(LogisticRegression,new_df,train_Y)

   Name  Survived
0     0         0
1     2         0
2     3         1
3     6         0
4     7         1
   Sex  Survived
0    0  0.742038
1    1  0.188908
   Ticket  Survived
0       0  1.000000
1       1  0.666667
2       2  0.000000
3       5  1.000000
4       6  1.000000
   Cabin  Survived
0      0       0.0
1      2       0.0
2      3       1.0
3      5       0.0
4      6       1.0
   Embarked  Survived
0         0  0.553571
1         1  1.000000
2         2  0.389610
3         3  0.336957
   Name_mean  Sex_mean  Ticket_mean  Cabin_mean  Embarked_mean
0          0  0.188908          0.0    0.299854       0.336957
1          1  0.742038          1.0    1.000000       0.553571
2          1  0.742038          1.0    0.299854       0.336957
3          1  0.742038          0.5    0.500000       0.336957
4          0  0.188908          0.0    0.299854       0.336957
accuracy:1.0
time:0.019947052001953125
